# CT-CBM 

---
Goal of the notebook: .

Inputs of the notebook:
- 

Output of the notebook:
- prediction + interpretation layer for text classification.


Takeaways: 
- .
- .

In [1]:
# !pip install -r requirements.txt

In [2]:
import sys
sys.path.append('../../run_experiments/')
sys.path.append('../../run_experiments/scripts')
sys.path.append('../../run_experiments/models')
sys.path.append('../../run_experiments/data')

import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import json
import glob
import re

import torch
import torch.nn as nn
from datasets import load_dataset
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset, Subset


# model for CBM
# import fonction for getting PLM and tokenizer
from models.utils import load_model_and_tokenizer

from concepts_discovery_utils import extract_target_words, create_context_window, load_model, run_concepts_discovery


**Delete awkward warning**

In [3]:
import logging
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

# 0. autoreload

In [4]:
#code for autoreload script associated with jupyter notebook
%load_ext autoreload
%autoreload 2

# 1.SETUP	 ENVIRONMENT VARIABLES

In [5]:
# import config
from load_config import load_config

model_name = 'bert-base-uncased'    # 'bert-base-uncased' ou 'deberta-large' or 'gemma'
dataset    = 'agnews'               # 'movies' / 'agnews' / 'dbpedia' / 'medical'/ 'ledgar'/ n24news
annotation = 'C3M'       # 'C3M' ou 'our_annotation' ou 'combined_annotation'
config = load_config(model_name, dataset)
config.annotation = annotation

# 2. Data Loading and Preparation


In [6]:
# import the data
from data_agnews import prepare_agnews_data

In [7]:
train_loader, test_loader, val_loader, train_df, val_df, test_df = prepare_agnews_data(config)

Chargement des fichiers sauvegardés...


# 3. Launching experimentation

In [8]:
import os
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score
from transformers import pipeline
from concepts_discovery_utils import (
    extract_target_words, create_context_window,
    load_model, run_concepts_discovery, calculate_macro_concept_frequencies,
    update_concept_frequencies, find_most_frequent_macro_concept,
    
 ) #summarize_concepts, 
from attribution_utils import (
    process_data_in_batches, get_example, example_attribution,
    split_dataloader
)
import json

# Ranking concepts 
from ranking_utils import rank_macro_concepts, most_k_important_macro_concepts, get_concept_at_rank, randomize_scores


# Launch experimentation for OUR pipeline

In [9]:
if config.model_name == 'gemma':
    from full_pipeline_gemma import JointResidualFittingModel
    from models.jointCBMv2_gemma import JointModel
else:
    from full_pipeline import JointResidualFittingModel
    from models.jointCBMv2 import JointModel

### 1. Load the joint model

In [10]:
embedder_model, embedder_tokenizer, ModelXtoCtoY_layer, _ = load_model_and_tokenizer(config, n_concepts = 1)
CBM_joint = JointModel(embedder_model, embedder_tokenizer, ModelXtoCtoY_layer, config, train_loader, val_loader)


### 2.call of the residual fitting part for pipeline below


In [11]:
from models.utils import RidgeLinearLayer

# residual fitting part for pipeline below
linear_layer = RidgeLinearLayer(config.dim, config.num_labels, l2_lambda=0.01)
linear_layer.to(config.device)

CBM_joint.linear_layer = linear_layer

### 3. Instancie the pipeline class

In [12]:
joint_residual_model = JointResidualFittingModel(
    joint_model = CBM_joint,
    linear_layer = linear_layer, 
    discovery_model = None, 
    discovery_tokenizer = None,
    config = config
    )

### 4. launch the pipeline

##  new dataset C3M

In [13]:
# # import the dataframe
from prepare_data import prepare_data_from_csv
df_aug_train, df_aug_test = prepare_data_from_csv(annotation  = config.annotation, config = config, return_test = True)

# # dataloaders
from concepts_bank_utils import create_dataloader
notre_loader_train = create_dataloader(df_aug_train, embedder_tokenizer, config.max_len, config.batch_size)
notre_loader_test = create_dataloader(df_aug_test, embedder_tokenizer, config.max_len, config.batch_size)

# Ablation study : random importance score launch 10 times

In [ ]:
import shutil
from pathlib import Path
import glob

# 1) Répertoires et noms
base_ckpt = Path(config.SAVE_PATH) / "blue_checkpoints" / config.model_name / "cavs" / config.cavs_type
our_ckpt  = Path(config.SAVE_PATH) / "blue_checkpoints" / config.model_name / "Our_CBM_joint"
our_ckpt.mkdir(parents=True, exist_ok=True)

for run in range(10):
    print(f"\n=== RUN {run} ===")
    # 2) Copier le coverage_evolution MJ_{run}.pkl vers le nom standard attendu
    src_pkl = base_ckpt / f"sorted_macro_concepts_coverage_MJ_{config.annotation}_{config.agg_mode}_{config.agg_scope}_{run}.pkl"
    dst_pkl = base_ckpt / f"sorted_macro_concepts_coverage_MJ_{config.annotation}_{config.agg_mode}_{config.agg_scope}.pkl"
    shutil.copy(src_pkl, dst_pkl)

    # TODO: delete this lien after
    config.num_epochs = 10
    
    # 3) Créer un dossier pour ce run
    run_dir = our_ckpt / f"run_{run}"
    run_dir.mkdir(parents=True, exist_ok=True)

    # 4) Instancier et lancer la pipeline
    from models.jointCBMv2 import JointModel
    from models.utils       import RidgeLinearLayer
    embedder_model, embedder_tokenizer, ModelXtoCtoY_layer, _ = load_model_and_tokenizer(
        config, n_concepts=1
    )
    CBM_joint = JointModel(
        embedder_model,
        embedder_tokenizer,
        ModelXtoCtoY_layer,
        config,
        train_loader,
        val_loader
    )
    linear_layer = RidgeLinearLayer(config.dim, config.num_labels, l2_lambda=0.01).to(device)
    joint_residual_model = JointResidualFittingModel(
        joint_model=CBM_joint,
        linear_layer=linear_layer,
        discovery_model=None,
        discovery_tokenizer=None,
        config=config
    )

    joint_residual_model.run_full_pipeline_tcavs_strategy(
        train_loader=notre_loader_train,
        val_loader=val_loader,
        test_loader=notre_loader_test,
        num_iterations=10,
        num_epochs_residual_layer=5,
        cavs_type_arg=config.cavs_type,
        strategy="new_heuristique_MJ",
        coverage_threshold=95
    )

    strategy="new_heuristique_MJ"
    
    # 5) Copier tous les .pth intermédiaires dans run_dir
    #    ils sont déposés dans our_ckpt, on glob par pattern strategy+iteration
    pattern = (
        our_ckpt /
        f"{config.model_name}_*.pth"
    )
    for src in glob.glob(str(pattern)):
        shutil.copy(src, run_dir / Path(src).name)
    
    pattern = (
        our_ckpt /
        f"{config.model_name}_*.pth"
    )
    for src in glob.glob(str(pattern)):
        shutil.move(src, run_dir / Path(src).name)
    
    # 6) Copier le JSON de perf
    src_perf = our_ckpt / f"{config.model_name}_performance_{strategy}.json"
    dst_perf = run_dir / f"{config.model_name}_performance_{strategy}_run{run}.json"
    shutil.move(src_perf, dst_perf)
    
    print(f"✅ Run {run} terminé. Tous les artefacts sont dans {run_dir}")

Moyenne +- ecart type

In [ ]:

# recalculer tous les test accuracy : cassetête chinois de ouf
model_name = 'bert-base-uncased'
dataset = 'movies'

if model_name == 'bert-base-uncased':
    if dataset == 'movies':
        from config_movies import Config as Config_movies
        config = Config_movies()
        # import the data
        from data_movies import prepare_movies_data
        train_loader, test_loader, val_loader, train_df, val_df, test_df = prepare_movies_data(config)

    elif dataset =='agnews':
        from config_agnews import Config as Config_agnews 
        config = Config_agnews()
        # import the data
        from data_agnews import prepare_agnews_data
        train_loader, test_loader, val_loader, train_df, val_df, test_df = prepare_agnews_data(config)

    elif dataset == 'dbpedia':
        from config_dbpedia import Config as Config_dbpedia 
        config = Config_dbpedia()
        # import the data
        from data_dbpedia import prepare_dbpedia_data
        train_loader, test_loader, val_loader, train_df, val_df, test_df = prepare_dbpedia_data(config)

    # elif dataset =='medical':
    #     from config_medical import Config as Config_medical
    #     config = Config_medical()
    #     # import the data
    #     from data_medical import prepare_medical_data
    #     train_loader, test_loader, val_loader, train_df, val_df, test_df = prepare_medical_data(config)

    else :
        print('enter a valid dataset name')
        pass




# charger les données 
from concepts_bank_utils import create_dataloader

embedder_model, embedder_tokenizer, ModelXtoCtoY_layer, _ = load_model_and_tokenizer(config, n_concepts = 1)

# charger les données de train
df_aug_train = pd.read_csv(f"{config.SAVE_PATH_CONCEPTS}/df_with_topics_v4_C3M.csv")

# adapt lines of code above to replace train_data and test_data by df_aug_train & df_aug_test
df_aug_train['text'] = df_aug_train['text'].astype(str).str.strip()

if(df_aug_train['label'].dtype != int):
    df_aug_train['label'] = df_aug_train["label"].astype(str).str.strip()

#dataset length
print("df_aug_train", len(df_aug_train))
print(df_aug_train['label'].unique())

# import json file
with open(f"{config.SAVE_PATH_CONCEPTS}/dictionary_{config.DATASET}.json", "r") as f:
    caption_to_number = json.load(f)

print(caption_to_number)

if(df_aug_train['label'].dtype != int):
    df_aug_train = df_aug_train[df_aug_train["label"].isin(caption_to_number.keys())]
    df_aug_train["label"] = df_aug_train["label"].map(caption_to_number)

print("df_aug_train", len(df_aug_train))
print(df_aug_train['label'].unique())

# determine total number of concepts
columns_CBM = [col for col in df_aug_train.drop(columns=['Unnamed: 0','text','label']) if 'dummy' in col]
columns_C3M = [col for col in df_aug_train.drop(columns=['Unnamed: 0','text','label']) if not 'dummy' in col]
n_concepts = len(columns_C3M)
print("..n_concepts", n_concepts)
print(columns_C3M)

# test on C3M smaller dataset (1000 instances) for concept accuracy
df_aug_test = pd.read_csv(f"{config.SAVE_PATH_CONCEPTS}/df_with_topics_v4_test_C3M.csv")

# adapt lines of code above to replace train_data and test_data by df_aug_train & df_aug_test
df_aug_test['text'] = df_aug_test['text'].astype(str).str.strip()
if(df_aug_test['label'].dtype != int):
    df_aug_test['label'] = df_aug_test["label"].astype(str).str.strip()

#dataset length
print("df_aug_test", len(df_aug_test))
print(df_aug_test['label'].unique())

print(caption_to_number)

if(df_aug_test['label'].dtype != int):
    df_aug_test = df_aug_test[df_aug_test["label"].isin(caption_to_number.keys())]
    df_aug_test["label"] = df_aug_test["label"].map(caption_to_number)

print("df_aug_test", len(df_aug_test))
print(df_aug_test['label'].unique())

# clean column names
df_aug_test.columns = ["dummy_"+col.replace("\n", "").strip() if col in columns_C3M else col.replace("\n", "").strip() for col in df_aug_test]

# clean types to int for "missing values"
for col in [col for col in df_aug_test.columns if (df_aug_test[col].dtype == 'O') and (col !='text') and (col!='label')]:
   # df_aug_test[col] = df_aug_test[col].apply(safe_int_convert)
    df_aug_test[col] = df_aug_test[col].apply(lambda x: int(x) if str(x).isdigit() else 0)

# dataloaders
notre_loader_test = create_dataloader(df_aug_test, embedder_tokenizer, config.max_len, config.batch_size)

# 1 recuperer le meilleur modèle charger 

# 1) Pattern pour trouver tous les JSON de performance
pattern = f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/Our_CBM_joint/run_*/*.json"
files = sorted(glob.glob(pattern))

# 2) Extraction des métriques
records = []
for path in files:
    # on récupère le numéro de run depuis le nom de dossier
    m = re.search(r"run_(\d+)", path)
    run_id = int(m.group(1)) if m else None

    # on ouvre le JSON
    with open(path, "r") as f:
        data = json.load(f)
    if len(data['test_metrics']) > 1: 
        from models.jointCBMv2 import JointModel
        
        device = config.device
        iteration = len(data['test_metrics']) - 1
        n_concept_start = len(data['concepts_discovered_by_iteration'])
        # load le modèle
        from models.jointCBMv2 import JointModel
        embedder_model, embedder_tokenizer, ModelXtoCtoY_layer, _ = load_model_and_tokenizer(config, n_concepts = n_concept_start)
        strategy = "new_heuristique_MJ"
        PATH_embedder = f"{config.SAVE_PATH}blue_checkpoints/{config.model_name}/Our_CBM_joint/run_{run_id}/{config.model_name}_embedder_state_dict_{strategy}_{iteration}.pth"
        PATH_ModelXtoCtoY_layer = f"{config.SAVE_PATH}blue_checkpoints/{config.model_name}/Our_CBM_joint/run_{run_id}/{config.model_name}_ModelXtoCtoY_layer_state_dict_{strategy}_{iteration}.pth"
        
        
        embedder_model.load_state_dict(torch.load(PATH_embedder))
        ModelXtoCtoY_layer.load_state_dict(torch.load(PATH_ModelXtoCtoY_layer))
        
        embedder_model.to(device)
        embedder_model.eval()
        
        CBM_joint_ = JointModel(embedder_model, embedder_tokenizer, ModelXtoCtoY_layer, config, None, None)
        CBM_joint_.concepts_name = data['concepts_discovered_by_iteration']
        print(".._model_.cavs.shape", len(CBM_joint_.concepts_name))
        print("..n_model_.classifier", CBM_joint_.ModelXtoCtoY_layer)

        # evaluate le model sur test_loader

        test_metrics_acc = CBM_joint_.evaluate_model(test_loader, 'test', metrics_on_concepts = False)
        test_metrics_metrics = CBM_joint_.evaluate_model(notre_loader_test, 'test', metrics_on_concepts = True)

        # recuperer le task_global_accuracy_ d'ici

        acc = test_metrics_acc["task_global_accuracy"]
        f1  = test_metrics_metrics["mean_concept_f1_score"]
        n_concepts = n_concept_start
        
    else:
        # on suppose que `data['test_metrics']` est une liste de dicts
        # et qu’on s’intéresse à la dernière itération
        last = data["test_metrics"][-1]
        acc = last["task_global_accuracy"]
        f1  = last["mean_concept_f1_score"]
        n_concepts = len(data['concepts_discovered_by_iteration'])


    records.append({
        "run": run_id,
        "test_acc": acc,
        "mean_concept_f1": f1,
        "n_concepts": n_concepts
    })

# 3) Construction du DataFrame
df = pd.DataFrame(records).sort_values("run").reset_index(drop=True)
print(df)

# 4) Calcul de la moyenne ± écart-type
acc_mean, acc_std = df["test_acc"].mean(), df["test_acc"].std()
f1_mean,  f1_std  = df["mean_concept_f1"].mean(), df["mean_concept_f1"].std()
n_concepts_mean,  n_concepts_std  = df["n_concepts"].mean(), df["n_concepts"].std()

print(f"\nTest Accuracy : {100*acc_mean:.2f} ± {100*acc_std:.2f}")
print(f"Mean Concept F1: {100*f1_mean:.2f} ± {100*f1_std:.2f}")
print(f"Mean Number of concepts: {n_concepts_mean:.2f} ± {n_concepts_std:.2f}")



# fin



